# 训练与生成
终于，我们来到了第四部分Training loop和Generating text，我们将最终整合迄今为止构建的主要组件：分词后的数据、模型和优化器。



## 5.1 数据加载（Data Loader）
分词后的数据（例如你在 tokenizer_experiments 中准备的数据）是一个单一的令牌序列 $x = (x_1, \cdots, x_n)$。尽管原始数据可能由多个独立的文档组成（例如不同的网页或源代码文件），常见的做法是将所有这些文档连接成一个单一的令牌序列，并在它们之间添加一个分隔符（例如 token）。

数据加载器会将这个长序列转换为一批批的数据流，其中每个批次包含 $B$ 个长度为 $m$ 的序列，以及对应的下一个令牌序列（长度也为 $m$）。例如，当 $B = 1,m = 3$ 时，$([x_2, x_3, x_4], [x_3, x_4, x_5])$ 就是一个可能的批次。

以这种方式加载数据在多个方面简化了训练过程。首先，任何满足 1 ≤ i < n − m 的位置 i 都可以生成一个有效的训练序列，因此序列的采样非常简单。由于所有训练序列长度相同，无需进行填充操作，这提高了硬件利用率（同时也可以增大批次大小 B）。

最后，我们也不必一次性将整个数据集全部加载到内存中即可进行训练样本的抽取，从而更容易处理那些可能无法完全放入内存的大型数据集。

#### 问题（data_loading）：实现数据加载
交付要求：编写一个函数，该函数接收一个 numpy 数组 x（包含令牌 ID 的整数数组）、batch_size、context_length 和一个 PyTorch 设备字符串（例如 'cpu' 或 'cuda:0'），并返回一对张量：采样的输入序列及其对应的下一个令牌目标。这两个张量的形状都应为 (batch_size, context_length)，包含令牌 ID，并且都应放置在指定的设备上。

> **低资源/降级训练提示：在 CPU 上进行数据加载**
> 
>如果你计划在 CPU 上训练你的语言模型，你需要将数据移动到正确的设备上（同样地，之后你的模型也应使用相同的设备）。

* 如果使用 CPU，可使用设备字符串 'cpu'；

In [1]:
import numpy as np
import torch


def get_batch(
    x: np.ndarray, batch_size: int, context_length: int, device: str
) -> tuple[torch.Tensor, torch.Tensor]:
    """
    生成训练批次数据

    Args:
        x: 令牌ID的一维numpy数组
        batch_size: 批次大小
        context_length: 序列长度
        device: 目标设备 (如 'cpu' 或 'cuda:0')

    Returns:
        (inputs,targets): (输入序列, 下一个目标序列) (batch_size, context_length)
    """
    # 确保数据足够长 (至少需要 context_length+1 个令牌)
    assert len(x) > context_length + 1, "数据长度不足"

    # 随机生成起始位置 (避免越界)
    max_start = len(x) - context_length - 1
    starts = np.random.randint(0, max_start, size=batch_size)

    # 提取输入序列和目标序列,生成批次 (batch_size, context_length)
    inputs = np.stack([x[i : i + context_length] for i in starts])
    targets = np.stack([x[i + 1 : i + context_length + 1] for i in starts])

    # 转为PyTorch张量并移动到指定设备
    return (
        torch.tensor(inputs, dtype=torch.long, device=device),
        torch.tensor(targets, dtype=torch.long, device=device),
    )

# 生成示例数据 (1000个令牌ID)
data = np.random.randint(0, 100, size=1000)

# 获取批次 (batch_size=4, 序列长=8, 使用CPU)
inputs, targets = get_batch(data, 4, 8, 'cpu')

print(inputs.shape)  # torch.Size([4, 8])
print(targets.shape) # torch.Size([4, 8])
print(inputs.device) # cpu


torch.Size([4, 8])
torch.Size([4, 8])
cpu


如果数据集太大而无法全部加载到内存中怎么办？我们可以使用一个名为 mmap 的 Unix 系统调用，它能将磁盘上的文件映射到虚拟内存，并在访问对应内存位置时才懒加载文件内容。因此，你可以“假装”整个数据集已经在内存中了。NumPy 通过 np.memmap 提供了这一功能（或者在使用 np.load 时设置参数 mmap_mode='r'，前提是你之前是用 np.save 保存的数组），这会返回一个类数组对象，只有在你访问具体元素时才会按需加载数据。

在训练过程中从数据集（即 NumPy 数组）采样时，务必以内存映射模式加载数据集（通过 np.memmap 或 np.load 的 mmap_mode='r' 参数，具体取决于你保存数组的方式）。同时，请确保指定的 dtype 与你所加载的数组的原始数据类型一致。为确保安全，建议显式验证内存映射的数据是否正确（例如，检查数据中是否包含超出预期词表大小的非法 token ID 值）。



## 5.2 检查点（Checkpointing）
除了加载数据外，我们还需要在训练过程中保存模型。我们常常希望能够在训练中途意外停止后（例如由于任务超时、机器故障等）**恢复训练**。

即使一切顺利，我们也可能希望之后能够访问训练过程中的中间模型（例如，事后研究训练动态、从不同训练阶段的模型中生成样本等）。

一个检查点（checkpoint）应包含所有能够恢复训练所需的状态。最基本的是，我们必须能够恢复模型的权重。如果使用了带有状态的优化器（例如 AdamW），我们还需要保存优化器的状态（例如，AdamW 中的动量估计值）。

最后，为了能够恢复学习率调度，我们还需要知道训练停止时所处的迭代次数。对于 pytorch 每个 `nn.Module` 都有一个 `state_dict()` 方法，它会返回一个包含所有可学习参数的字典；之后我们可以通过对应的 `load_state_dict()` 方法恢复这些权重。

优化器（`torch.optim.Optimizer`）也同样支持 `state_dict()` 和 `load_state_dict()` 方法。

最后，`torch.save(obj, dest)` 可以将一个对象（例如，字典，其值中包含张量，也可以包含整数等普通 Python 对象）序列化并保存到文件（路径）或类文件对象中，之后可以通过 `torch.load(src)` 将其重新加载回内存。

#### 问题（checkpointing）：实现模型检查点保存与加载
实现以下两个函数，用于保存和加载模型检查点：

`def save_checkpoint(model, optimizer, iteration, out)` 应将前三个参数中的所有状态保存到文件类对象 out 中。你可以使用模型和优化器的 state_dict 方法获取它们的相关状态，并使用 `torch.save(obj, out)` 将 obj 保存到 out（PyTorch 支持路径或文件类对象）。一个常见的做法是让 obj 是一个字典，但你也可以使用任何格式，只要之后能够成功加载你的检查点即可。

该函数需要以下参数：

* model: torch.nn.Module
* optimizer: torch.optim.Optimizer
* iteration: int
* out: str | os.PathLike | typing.BinaryIO | typing.IO[bytes]

`def load_checkpoint(src, model, optimizer) `应从 src（路径或文件类对象）加载检查点，并恢复模型和优化器的状态。你的函数应返回保存在检查点中的迭代次数。你可以使用 `torch.load(src)` 来读取你在 save_checkpoint 中保存的内容，并通过模型和优化器的 `load_state_dict` 方法将它们恢复到之前的状态。

该函数需要以下参数：

* src: str | os.PathLike | typing.BinaryIO | typing.IO[bytes]
* model: torch.nn.Module
* optimizer: torch.optim.Optimizer

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim


def save_checkpoint(
    model: nn.Module, optimizer: optim.Optimizer, iteration: int, out: str
):
    """
    保存模型检查点

    Args:
        model: 要保存的模型
        optimizer: 优化器
        iteration: 当前训练迭代次数
        out: 保存目标 (文件路径/文件对象)
    """
    # 保存模型状态、优化器状态和迭代次数
    torch.save(
        {
            "model": model.state_dict(),
            "optimizer": optimizer.state_dict(),
            "iteration": iteration,
        },
        out,
    )


def load_checkpoint(src: str, model: nn.Module, optimizer: optim.Optimizer) -> int:
    """
    加载模型检查点

    Args:
        src: 检查点源 (文件路径/文件对象)
        model: 待恢复的模型
        optimizer: 待恢复的优化器

    Returns:
        保存的迭代次数
    """
    # 加载检查点数据
    checkpoint = torch.load(src)

    # 恢复模型和优化器状态
    model.load_state_dict(checkpoint["model"])
    optimizer.load_state_dict(checkpoint["optimizer"])

    return checkpoint["iteration"]


class SimpleModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(10, 5)

    def forward(self, x):
        return self.linear(x)


# 创建模型和优化器实例
model = SimpleModel()
optimizer = optim.SGD(model.parameters(), lr=0.01)
# 保存检查点
save_checkpoint(model, optimizer, iteration=1000, out="./out/ckpt.pt")

# 加载检查点
iteration = load_checkpoint("./out/ckpt.pt", model, optimizer)
print(f"恢复到迭代次数: {iteration}")  # 输出: 恢复到迭代次数: 1000

恢复到迭代次数: 1000


C:\Users\10611\AppData\Local\Temp\ipykernel_29232\709772644.py:42: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(src)
